# 解壓縮

In [ ]:
!rm -rf dataset_xy
!rm -rf output
!unzip -q road_following_dataset_xy_2024-11-03_12-08-48.zip

# 函式庫

In [ ]:
!pip uninstall scipy




In [4]:
!pip install scipy

Looking in indexes: https://mirror.sjtu.edu.cn/pypi/web/simple



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\Chino\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
!pip3 --version
!pip3 install torch==2.5.0 torchvision==0.20.0 torchaudio==2.5.0 --index-url https://download.pytorch.org/whl/cu121

pip 21.3.1 from C:\Users\Chino\Documents\Methylation_analysis_prostate_cancer\venv\lib\site-packages\pip (python 3.10)

Looking in indexes: https://download.pytorch.org/whl/cu121


You should consider upgrading via the 'C:\Users\Chino\Documents\Methylation_analysis_prostate_cancer\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader

import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
import torch.nn.functional as F
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
import torch.nn as nn


# 檔案格式解碼


In [32]:
import glob
import os
import PIL.Image
import numpy as np
import torch
import torchvision.transforms as transforms

def get_x(path, width):
    """Gets the x value from the image filename"""
    return (float(int(path.split("_")[1])) - width / 2) / (width / 2)

def get_y(path, height):
    """Gets the y value from the image filename"""
    return (float(int(path.split("_")[2])) - height / 2) / (height / 2)

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        width, height = image.size
        x = float(get_x(os.path.basename(image_path), width))
        y = float(get_y(os.path.basename(image_path), height))
        
        # 裁切掉上半部 40%，只保留下半部的 60%
        cropped_image = image.crop((0, int(height * 0.4), width, height))  # 保留從 40% 開始到底部的部分
        
        # 水平翻轉圖片（若隨機觸發）
        if float(np.random.rand(1)) > 0.5 and self.random_hflips:
            cropped_image = transforms.functional.hflip(cropped_image)
            x = -x
        
        # 應用顏色抖動
        cropped_image = self.color_jitter(cropped_image)
        
        # 調整圖片大小至 224x134
        cropped_image = transforms.functional.resize(cropped_image, (134, 224))
        
        # 轉換為 tensor 並進行標準化
        image_tensor = transforms.functional.to_tensor(cropped_image)
        image_tensor = transforms.functional.normalize(image_tensor, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image_tensor, torch.tensor([x, y]).float()
    
# 建立資料集實例
dataset = XYDataset('1600-v4', random_hflips=False)
print(dataset[0][1])


tensor([-0.9464,  0.7143])


C:\Users\Chino\AppData\Local\Temp\ipykernel_87060\1113085698.py:39: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5 and self.random_hflips:


# 區分訓練集


In [33]:
test_percent = 0.2
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

# 提早結束


In [34]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True


# 訓練


In [30]:
import torch
print(torch.__version__)
print(torch.version.cuda)  # 查看 PyTorch 所使用的 CUDA 版本
print(torch.cuda.is_available())  # 檢查是否可使用 CUDA


2.5.0+cu121
12.1
True


In [35]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import scipy
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)
NUM_EPOCHS = 50
BEST_MODEL_PATH = 'resnet18_1600_v4.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += float(loss)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

C:\Users\Chino\AppData\Local\Temp\ipykernel_87060\1113085698.py:39: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5 and self.random_hflips:


0.111676, 0.025954
0.029500, 0.027071
0.022834, 0.014071
0.012432, 0.011188
0.011674, 0.015289
0.010206, 0.015112
0.009663, 0.009750
0.009204, 0.010560
0.011167, 0.009727
0.008948, 0.008643
0.005645, 0.007725
0.004614, 0.013742
0.004643, 0.005891
0.006342, 0.008669
0.005899, 0.010939
0.004485, 0.011851
0.005109, 0.006548
0.004945, 0.008827
0.004991, 0.006896
0.004830, 0.009192
0.004732, 0.006903
0.004964, 0.011957
0.004238, 0.008222
0.003573, 0.007792
0.002915, 0.008964
0.002557, 0.006004
0.002872, 0.006033
0.002657, 0.007273
0.003385, 0.006103
0.003399, 0.014701
0.035459, 0.045124
0.015996, 0.011282
0.009927, 0.010069
0.009273, 0.011890
0.006661, 0.007899
0.006020, 0.009209
0.005370, 0.007762
0.005119, 0.013006
0.004343, 0.006735
0.003754, 0.007399
0.003033, 0.006556
0.002673, 0.007999
0.002733, 0.007205
0.002915, 0.006168
0.003823, 0.007797
0.003248, 0.008567
0.003509, 0.009106
0.003122, 0.006138
0.003038, 0.007904
0.002717, 0.007519


In [10]:
import time
import torch
import torchvision.transforms as transforms
import cv2
import os
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import torchvision.models as models

# 設定資料夾路徑
image_folder = "T"
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# 設定 widget 以顯示圖片
widget_width = 224
widget_height = 224
image_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)
display(image_widget)

# 設定裝置 (若有 GPU 可用)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# 加載模型
model = models.resnet18()
model.fc = torch.nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("best_steering_model_xy.pth", map_location=device))
model = model.to(device)
model.eval()  # 將模型設定為推論模式

# 圖片預處理和顯示函數
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4691, 0.4032, 0.4579], [0.1740, 0.1485, 0.1688])
])

def bgr8_to_jpeg(image):
    _, jpeg = cv2.imencode('.jpg', image)
    return jpeg.tobytes()

def process_image(image_path):
    # 讀取圖片並進行裁切
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, _ = image.shape
    
    # 裁切圖片的下部 60%
    cropped_image = image[int(height * 0.4):, :, :]  # 保留從 40% 開始到高度底部的部分

    # 調整裁切後的圖片大小為 224x134
    cropped_image = cv2.resize(cropped_image, (224, 134))
    
    # 預處理圖片
    input_tensor = transform(cropped_image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(input_tensor)
        x, y = output[0].cpu().numpy()
    print(x,y/2)
    # 映射預測的 (x, y) 到圖片的像素坐標，假設 x 和 y 是 [0, 1] 範圍內的預測
    x_pixel = int(x * 224 / 2 + 224 / 2)
    y_pixel = int(y * 134 / 2 + 134 / 2)
    
    # 在圖片上繪製預測結果
    display_image = cv2.cvtColor(cropped_image, cv2.COLOR_RGB2BGR)  # 確保格式正確
    cv2.circle(display_image, (x_pixel, y_pixel), 5, (0, 255, 0), -1)  # 綠色點表示預測位置
    
    # 更新 widget 顯示處理後的圖片
    image_widget.value = bgr8_to_jpeg(display_image)

# 對資料夾中的每張圖片進行處理
for image_file in image_files:
    process_image(image_file)
    time.sleep(1)  # 暫停以觀察每張圖片的結果


Image(value=b'', format='jpeg', height='224', width='224')

cuda


C:\Users\Chino\AppData\Local\Temp\ipykernel_88864\199114618.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_steering_model_xy.pth

-0.95135546 0.29102370142936707
-0.98095137 0.29247230291366577
-0.9993132 0.3465510606765747
-0.93912655 0.2902553677558899
-0.874291 0.35392051935195923
-0.90726703 0.3767988085746765
-0.85764605 0.3805960714817047
-0.9011937 0.2968124449253082
-0.87809366 0.2949519157409668
-0.78742516 0.39971238374710083
-0.8487449 0.27218806743621826
-0.8514834 0.26460787653923035
-0.891951 0.29235756397247314
-0.87646466 0.34732359647750854
-0.85839826 0.3625093102455139
-0.91548085 0.35081619024276733
-0.8829027 0.3716273605823517
-0.9000686 0.3520094156265259
-0.8892612 0.3864104449748993
-0.8520264 0.39674630761146545
-0.8803156 0.28846293687820435
-0.81127465 0.29193976521492004
-0.80800766 0.344381719827652
-0.860281 0.353018581867218
-0.89076424 0.3789635896682739
-0.8686025 0.3747183680534363
-0.7933407 0.3995403051376343
-0.7388597 0.3099821209907532
-0.8394382 0.3285602331161499
-0.88862664 0.38699641823768616
-0.8863674 0.37544193863868713
-0.7851975 0.32652756571769714
-0.8592923 0.340

In [1]:
!pip install torch-pruning


Looking in indexes: https://mirror.sjtu.edu.cn/pypi/web/simple


You should consider upgrading via the 'C:\Users\Chino\Documents\Methylation_analysis_prostate_cancer\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
import torch
import torch.nn as nn
from torchvision import models
import torch.nn.utils.prune as prune

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 初始化 ResNet-18 模型并调整最后一层以匹配您的任务（假设输出为2个类别）
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, 2)  # 根据您的任务调整输出类别数
model = model.to(device)

# 加载已训练的权重
model_path = "best_steering_model_xy.pth"  # 替换为您的模型文件路径
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()  # 设置模型为推理模式

# 定义剪枝函数
def prune_model(model, amount=0.2):
    """
    对模型的卷积层和全连接层应用剪枝。
    :param model: 要剪枝的模型
    :param amount: 剪枝比例，例如0.2表示剪掉20%的参数
    """
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')  # 移除剪枝的重参数，保留剪枝后的权重

# 应用剪枝
prune_model(model, amount=0.2)

# 测试剪枝后的模型
with torch.no_grad():
    test_input = torch.randn(1, 3, 224, 224).to(device)  # 根据您的输入尺寸调整
    output = model(test_input)
    print("Model output after pruning:", output)

# 保存剪枝后的模型
torch.save(model.state_dict(), "pruned_best_steering_model_xy.pth")
print("Pruned model saved as pruned_best_steering_model_xy.pth")


Using device: cuda


C:\Users\Chino\AppData\Local\Temp\ipykernel_64504\1342711999.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Model output after pruning: tensor([[-0.2217,  0.6112]], device='cuda:0')
Pruned model saved as pruned_best_steering_model_xy.pth
